In [11]:
!pip install azure.keyvault.secrets azure.cognitiveservices.speech azure.cognitiveservices.language.luis azure.keyvault azure.identity  msrest

In [12]:
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
import azure.cognitiveservices.speech as speechsdk
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from msrest.authentication import CognitiveServicesCredentials
import requests
import json
from pathlib import Path


In [23]:
def read_config():
    # Read config.json file in the same folder as this script
    # config_path = Path(__file__).parent.joinpath('config.json')
    # config = json.loads(config_path.read_text())
    return {"keyvault_url":"", "speech_secret_name":"", "speech_service_region":"", "luis_secret_name":"", "luis_app_id_secret_name":"", "luis_endpoint":"","tessel_address":""}

def get_keyvault_client(keyvault_uri):
    # credential = DefaultAzureCredential()
    # keyvault_client = SecretClient(vault_url=keyvault_uri, credential=credential)
    return "keyvault_client"

def get_secret(secret_name):
    # secret = keyvault_client.get_secret(secret_name).value
    # return secret
    pass

def get_speech_recognizer(secret_name, service_region):
    # Get key from keyvault
    # speech_key = get_secret(secret_name)

    # Get speech client
    speech_config = speechsdk.SpeechConfig(subscription="7df2c5c264e14389917e913ab03092bf", region="eastus")
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config)
    return speech_recognizer

def recognize_speech():
    # Recognize microphone input
    print("Begin speaking...")
    result = speech_recognizer.recognize_once()

    # Print result
    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print("Recognized: {}".format(result.text))
        return result.text
    elif result.reason == speechsdk.ResultReason.NoMatch:
        print("No speech could be recognized: {}".format(result.no_match_details))
    elif result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = result.cancellation_details
        print("Speech Recognition canceled: {}".format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print("Error details: {}".format(cancellation_details.error_details))

def get_luis_runtime_client(secret_name, luis_endpoint):
    # Get secrets from keyvault
    # luis_key = get_secret(secret_name)

    # Get runtime client
    # luis_credentials = CognitiveServicesCredentials(luis_key)
    luis_client = LUISRuntimeClient(endpoint="https://mayax.cognitiveservices.azure.com/", credentials="d7ba36bdda354894baa36f770615c17a")
    return luis_client

def interpret_command(query):
    # Perform prediction
    print('Performing prediction...')
    prediction_request = {'query': query}
    prediction_response = luis_client.prediction.get_slot_prediction(luis_app_id, 'Production', prediction_request)

    # Print results
    print('Top intent: {}'.format(prediction_response.prediction.top_intent))
    print('Intents: ')
    for intent in prediction_response.prediction.intents:
        print('\t{}'.format (json.dumps (intent)))
    print('Entities: {}'.format (prediction_response.prediction.entities))

    return prediction_response.prediction

    # Auth and setup
config = read_config()
keyvault_client = get_keyvault_client(config['keyvault_url'])
speech_recognizer = get_speech_recognizer(config['speech_secret_name'], config['speech_service_region'])
luis_app_id = "fd444780-ad29-4766-9aeb-e7b5246d52bf"
luis_client = get_luis_runtime_client(config['luis_secret_name'], config['luis_endpoint'])
url = config['tessel_address']

while True:
    voice_command = recognize_speech() # Speech to text
    prediction = interpret_command(voice_command) # Text to intention

    if 'LED' in prediction.entities:
        # Extract entities and intent
        leds = prediction.entities['LED']
        intent = prediction.top_intent

        # Assign the recognized intent to each LED mentioned
        body = {}
        for color in [x[0] for x in leds]:
            body[color] = intent

        # Post the command to the tessel
        response = requests.post(url, data=json.dumps(body))
        print(response.text)
    else:
        print('Could not identify a known LED in the command.')

    input('Press any key to issue another command or Ctrl+C to quit.')

Begin speaking...
Recognized: How the weather in Cairo.
Performing prediction...


AttributeError: 'str' object has no attribute 'signed_session'